## Code Ref. for Gui images > lens > pyqt5

> promt: 

- add to widgets.canvas logic n functionality of brush button

- [ai share](https://chatgpt.com/share/67080738-0650-8000-ba60-3d0250523a9a)

In [ ]:
# widgets.canvas
import os
from PyQt5.QtWidgets import QLabel, QMessageBox
from PyQt5.QtCore import QPoint, Qt, QRect
from PyQt5.QtGui import QPixmap, QPainter, QPen, QImage
from PIL import Image
from arh.utils.image_conversion import pil_to_qimage, qpixmap_to_pil

from arh.core.image_manager import ImageManager


class Canvas(QLabel):
    """Widget for handling image display, cropping, and brush drawing logic."""

    def __init__(self, parent):
        super().__init__(parent)
        self.parent = parent
        self.im_manager = ImageManager()

        self.drawing = False
        self.brush_active = False  # Whether the brush tool is active
        self.pen_color = Qt.black  # Default brush color
        self.pen_size = 5          # Default brush size
        self.last_point = QPoint()  # To store the last point for drawing lines

        self.original_image = None
        self.start_point = None
        self.end_point = None
        self.scale_factor = 1.0

        self.setPixmap(QPixmap())  # Start with a blank pixmap

    def activate_brush(self, activate=True):
        """Activate or deactivate the brush tool."""
        self.brush_active = activate

    def set_pen_color(self, color):
        """Set the color of the brush."""
        self.pen_color = color

    def set_pen_size(self, size):
        """Set the size of the brush."""
        self.pen_size = size

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            if self.brush_active:
                self.drawing = True
                self.last_point = event.pos()
            else:
                self.start_point = event.pos()
                self.end_point = event.pos()
            self.update()

    def mouseMoveEvent(self, event):
        if self.drawing and self.brush_active:
            # Draw with brush while moving the mouse
            painter = QPainter(self.pixmap())
            painter.setPen(QPen(self.pen_color, self.pen_size,
                           Qt.SolidLine, Qt.RoundCap, Qt.RoundJoin))
            painter.drawLine(self.last_point, event.pos())
            self.last_point = event.pos()
            self.update()
        elif self.start_point:
            self.end_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton and self.brush_active:
            self.drawing = False
        elif event.button() == Qt.LeftButton:
            self.end_point = event.pos()
            self.crop_image()

    def paintEvent(self, event):
        """Draw a rectangle when cropping or handle other paint events."""
        super().paintEvent(event)
        if self.start_point and self.end_point:
            painter = QPainter(self)
            painter.setPen(QPen(Qt.green, 2, Qt.SolidLine))
            rect = QRect(self.start_point, self.end_point)
            painter.drawRect(rect)

    # Other methods (load_image, resizeEvent, crop_image, etc.) remain unchanged

In [ ]:
# ui.photoshop
# Assuming you are using QPushButton for other tools
brush_button = QPushButton("Brush")
# Activates the brush tool
brush_button.clicked.connect(lambda: self.canvas.activate_brush(True))

# Add this button to your layout, tool panel, or toolbar
toolbar_layout.addWidget(brush_button)

In [ ]:
# Brush Settings Panel
# 3# Brush size slider
brush_size_slider = QSlider(Qt.Horizontal)
brush_size_slider.setMinimum(1)
brush_size_slider.setMaximum(20)
brush_size_slider.valueChanged.connect(
    lambda value: self.canvas.set_pen_size(value))

# Brush color picker (using QColorDialog for example)
color_button = QPushButton("Pick Color")
color_button.clicked.connect(
    lambda: self.canvas.set_pen_color(QColorDialog.getColor()))

### Promnt2

- logic of all QSliders inside widgets.frame_layout

In [ ]:
# widgets.frame_layout
from PyQt5.QtWidgets import QFrame, QVBoxLayout, QLabel, QSlider, QColorDialog
from PyQt5.QtCore import Qt


class FrameWithSliders(QFrame):
    """A frame that holds sliders for adjusting brush settings or image properties"""

    def __init__(self, canvas, parent=None):
        super().__init__(parent)
        self.canvas = canvas

        # Create a vertical layout for the frame
        self.frame_layout = QVBoxLayout()

        # Add sliders for various controls

        # Brush size slider
        self.brush_size_label = QLabel("Brush Size")
        self.brush_size_slider = QSlider(Qt.Horizontal)
        self.brush_size_slider.setMinimum(1)
        self.brush_size_slider.setMaximum(20)
        self.brush_size_slider.setValue(5)  # Set default value
        self.brush_size_slider.valueChanged.connect(self.update_brush_size)

        # Opacity slider (example for another property)
        self.opacity_label = QLabel("Opacity")
        self.opacity_slider = QSlider(Qt.Horizontal)
        self.opacity_slider.setMinimum(0)
        self.opacity_slider.setMaximum(100)
        self.opacity_slider.setValue(100)  # Set default value
        self.opacity_slider.valueChanged.connect(self.update_opacity)

        # Image contrast slider
        self.contrast_label = QLabel("Image Contrast")
        self.contrast_slider = QSlider(Qt.Horizontal)
        self.contrast_slider.setMinimum(0)
        self.contrast_slider.setMaximum(200)
        self.contrast_slider.setValue(100)  # Normal contrast as default
        self.contrast_slider.valueChanged.connect(self.update_contrast)

        # Add labels and sliders to the layout
        self.frame_layout.addWidget(self.brush_size_label)
        self.frame_layout.addWidget(self.brush_size_slider)
        self.frame_layout.addWidget(self.opacity_label)
        self.frame_layout.addWidget(self.opacity_slider)
        self.frame_layout.addWidget(self.contrast_label)
        self.frame_layout.addWidget(self.contrast_slider)

        # Set layout for the frame
        self.setLayout(self.frame_layout)

    def update_brush_size(self, value):
        """Update the brush size in the canvas"""
        self.canvas.set_pen_size(value)

    def update_opacity(self, value):
        """Update the opacity (transparency) of the drawing/painting"""
        # Example logic: update canvas opacity if needed
        # You can adjust the canvas opacity or any other property here
        pass

    def update_contrast(self, value):
        """Update the image contrast in the canvas"""
        # Apply contrast adjustment logic here (if needed in the canvas)
        pass

In [ ]:
# connnect with photoshop.py
from PyQt5.QtWidgets import QMainWindow, QWidget, QHBoxLayout


class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        # Main widget and layout
        self.main_widget = QWidget()
        self.main_layout = QHBoxLayout(self.main_widget)

        # Add canvas (where the image is drawn or manipulated)
        `self.canvas = Canvas(self)`

        # Add the frame with sliders
        `self.sliders_frame = FrameWithSliders(self.canvas)`

        # Add both widgets to the main layout
        self.main_layout.addWidget(self.canvas)
        self.main_layout.addWidget(self.sliders_frame)

        # Set main layout and widget
        self.setCentralWidget(self.main_widget)
        self.setWindowTitle("Image Editor with Sliders")

        # Window size
        self.resize(800, 600)